# Install the Watson IoT Platform Python SDK

In [1]:
pip install wiotp.sdk

     |████████████████████████████████| 96 kB 9.1 MB/s  eta 0:00:01
     |████████████████████████████████| 99 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 3.9 MB/s  eta 0:00:01
  Created wheel for wiotp.sdk: filename=wiotp_sdk-0.11.0-py3-none-any.whl size=97104 sha256=66f991be59fe4c5c51d14980ef8d6f8c3c99f3b8f233f5d7ee99fe6693b8ac2a
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/56/f1/4b/2d7bc55ae8f9ef6e175c2c8be828fcd108b8c4b8c5ff613136
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62116 sha256=552f902820495896354180e5f871a4fe7a0a2a170dbf95582b8d33b14106de43
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/d0/bf/ac/2b3f43f8c6fcd0f4ba5395397458c521eb0b52d33b574a5a40
Successfully built wiotp.sdk paho-mqtt
Note: you may need to restart the kernel to use updated packages.


# Import the (few) required packages

In [2]:
import wiotp.sdk.device
import pandas as pd
import time
import argparse
from datetime import datetime

# Capture your WIoTP Device Configuration

## OPTION A - For MONITOR SaaS

In [3]:
# OPTION A: For MONITOR SaaS

import wiotp.sdk.device

myConfig = { 
    "identity": {
        "orgId": "YOUR_ORGID",
        "typeId": "YOUR_Device",
        "deviceId": "YOUR_Device001"
    },
    "auth": {
        "token": "YOUR_AUTHENTICATION_TOKEN"
    }
}

## OPTION B - For MONITOR part of MAS (Maximo Application Suite)

In [3]:
# OPTION B: For MONITOR on MAS (Maximop Application Suite)

import wiotp.sdk.device

# Provide the Device's configuration
# Refer to for details on how to get all this info
myConfig = { 
    "identity": {
        "orgId": "YOUR_ORGID",
        "typeId": "YOUR_Device",
        "deviceId": "YOUR_Device001"
    },
    "auth": {
        "token": "YOUR_AUTHENTICATION_TOKEN"
    },
    "options": {
        "domain": "iot.maximoabc.yourcloud.info",
        "http":{
            "verify": "True"
        },
        "mqtt":{
            "port":443,
            "caFile": "/project_data/data_asset/iot-maximoabc-yourcloud-info-chain.pem"
        }
    }
}

# Connect your Device to WIoTP

In [4]:
deviceCli = wiotp.sdk.device.DeviceClient(config=myConfig)
print(myConfig)
deviceCli.connect()

2022-01-04 01:51:20,109   wiotp.sdk.device.client.DeviceClient  INFO    Connected successfully: d:YOUR_ORGID:YOUR_Device:YOUR_Device001


{'identity': {'orgId': 'YOUR_ORGID', 'typeId': 'YOUR_Device', 'deviceId': 'YOUR_Device001'}, 'auth': {'token': 'YOUR_AUTHENTICATION_TOKEN'}, 'options': {'domain': 'iot.maximoabc.yourcloud.info', 'http': {'verify': 'True'}, 'mqtt': {'port': 443, 'caFile': '/project_data/data_asset/iot-maximoabc-yourcloud-info-chain.pem', 'transport': 'tcp', 'cleanStart': False, 'sessionExpiry': 3600, 'keepAlive': 60}, 'logLevel': 20}}


# Read the '0_1080_All.csv' Pattern File

In [5]:
monitor_patterns_readings = pd.read_csv('/project_data/data_asset/0_1080_All.csv',
                 index_col=False)
monitor_patterns_readings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       1080 non-null   int64  
 1   reading_1  1080 non-null   float64
 2   reading_2  1080 non-null   float64
 3   reading_3  1080 non-null   float64
 4   reading_4  1080 non-null   float64
 5   reading_5  1080 non-null   float64
 6   reading_6  1080 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 59.2 KB


# Send the 1080 rows of '0_1080_All.csv' data to your Device on WIoTP

In [6]:
for index, row in monitor_patterns_readings.iterrows():
    data = {'reading_1': row['reading_1'],
            'reading_2': row['reading_2'],
            'reading_3': row['reading_3'],
            'reading_4': row['reading_4'],
            'reading_5': row['reading_5'],
            'reading_6': row['reading_6']
            }
    print(data)

    parser = argparse.ArgumentParser()
    parser.add_argument("-E", "--event", required=False, default="event", help="type of event to send")
    parser.add_argument("-N", "--nummsgs", required=False, type=int, default=1, help="send this many messages before disconnecting")
    
    parser.add_argument("-D", "--delay", required=False, type=float, default=5, help="number of seconds between msgs")
    args, unknown = parser.parse_known_args()

    def myOnPublishCallback():
        print()

    success = deviceCli.publishEvent(args.event, "json", data, qos=0, onPublish=myOnPublishCallback)
    now = datetime.now()
    print("======SUCCESS======", now, "===Row:", index,"Success:", success)
    if not success:
        print("Not connected to WIoTP")

    time.sleep(args.delay)
    

deviceCli.disconnect()

{'reading_1': 10.85066167, 'reading_2': 10.01964812, 'reading_3': 20.15516953, 'reading_4': 20.87776326, 'reading_5': 10.04102436, 'reading_6': 12.94680491}
======SUCCESS====== 2022-01-04 01:52:23.098698 ===Row: 0 Success: True

{'reading_1': 10.59574274, 'reading_2': 10.80853745, 'reading_3': 20.87205928, 'reading_4': 20.10878476, 'reading_5': 10.17112379, 'reading_6': 10.32845991}
======SUCCESS====== 2022-01-04 01:52:28.122078 ===Row: 1 Success: True

{'reading_1': 10.07333239, 'reading_2': 10.48054819, 'reading_3': 20.02722103, 'reading_4': 20.46039852, 'reading_5': 10.35664339, 'reading_6': 13.93976121}
======SUCCESS====== 2022-01-04 01:52:33.149557 ===Row: 2 Success: True

{'reading_1': 10.90152952, 'reading_2': 10.21132138, 'reading_3': 20.57671781, 'reading_4': 20.21359487, 'reading_5': 10.95080441, 'reading_6': 10.69124238}
======SUCCESS====== 2022-01-04 01:52:38.176823 ===Row: 3 Success: True

{'reading_1': 10.54842036, 'reading_2': 10.4896714, 'reading_3': 20.52216801, 'readi

{'reading_1': 10.52234547, 'reading_2': 10.36484044, 'reading_3': 20.08028765, 'reading_4': 20.04705393, 'reading_5': 10.11678553, 'reading_6': 13.1292419}
======SUCCESS====== 2022-01-04 01:55:23.951829 ===Row: 36 Success: True

{'reading_1': 10.20136899, 'reading_2': 10.00304212, 'reading_3': 20.48567865, 'reading_4': 20.37314922, 'reading_5': 10.14345842, 'reading_6': 10.45554465}
======SUCCESS====== 2022-01-04 01:55:28.971654 ===Row: 37 Success: True

{'reading_1': 10.80886107, 'reading_2': 10.74164976, 'reading_3': 20.25778963, 'reading_4': 20.24706856, 'reading_5': 10.511980300000001, 'reading_6': 13.12175555}
======SUCCESS====== 2022-01-04 01:55:33.998015 ===Row: 38 Success: True

{'reading_1': 10.76131408, 'reading_2': 10.02212994, 'reading_3': 20.06920683, 'reading_4': 20.58498817, 'reading_5': 10.98114737, 'reading_6': 10.19189523}
======SUCCESS====== 2022-01-04 01:55:39.024570 ===Row: 39 Success: True

{'reading_1': 10.59287702, 'reading_2': 10.70983011, 'reading_3': 20.76562

{'reading_1': 10.96233912, 'reading_2': 11.168484900000001, 'reading_3': 23.88836232, 'reading_4': 23.96212364, 'reading_5': 10.57013926, 'reading_6': 12.61549734}
======SUCCESS====== 2022-01-04 01:58:24.797104 ===Row: 72 Success: True

{'reading_1': 10.40360585, 'reading_2': 11.11193032, 'reading_3': 23.17512971, 'reading_4': 23.99251421, 'reading_5': 10.86178183, 'reading_6': 11.66367074}
======SUCCESS====== 2022-01-04 01:58:29.823957 ===Row: 73 Success: True

{'reading_1': 10.14727465, 'reading_2': 10.854824699999998, 'reading_3': 23.5403244, 'reading_4': 23.12849283, 'reading_5': 10.49158549, 'reading_6': 13.36904021}
======SUCCESS====== 2022-01-04 01:58:34.842659 ===Row: 74 Success: True

{'reading_1': 10.77896673, 'reading_2': 11.57355516, 'reading_3': 23.33930739, 'reading_4': 23.368385, 'reading_5': 10.27672537, 'reading_6': 11.58706602}
======SUCCESS====== 2022-01-04 01:58:39.869125 ===Row: 75 Success: True

{'reading_1': 10.007681800000002, 'reading_2': 11.58053225, 'reading_

{'reading_1': 10.23419331, 'reading_2': 11.83284581, 'reading_3': 25.06270413, 'reading_4': 25.04186712, 'reading_5': 20.1414503, 'reading_6': 12.62073144}
======SUCCESS====== 2022-01-04 02:01:25.673588 ===Row: 108
 Success: True
{'reading_1': 10.33164213, 'reading_2': 11.42243306, 'reading_3': 25.52773385, 'reading_4': 25.48888595, 'reading_5': 20.92472351, 'reading_6': 11.10433185}
======SUCCESS====== 2022-01-04 02:01:30.699375 ===Row: 109 Success: True

{'reading_1': 10.58394862, 'reading_2': 11.51120221, 'reading_3': 25.69741381, 'reading_4': 25.67096553, 'reading_5': 20.188368899999997, 'reading_6': 12.71003691}
======SUCCESS====== 2022-01-04 02:01:35.726010 ===Row: 110 Success: True

{'reading_1': 10.80605891, 'reading_2': 11.80461587, 'reading_3': 25.82966748, 'reading_4': 25.40522407, 'reading_5': 20.58918726, 'reading_6': 11.51992034}
======SUCCESS====== 2022-01-04 02:01:40.751664 ===Row: 111 Success: True

{'reading_1': 10.90678698, 'reading_2': 11.45640713, 'reading_3': 25.3

{'reading_1': 10.91532854, 'reading_2': 12.98391139, 'reading_3': 25.36291096, 'reading_4': 25.70346445, 'reading_5': 20.3743336, 'reading_6': 12.23779786}
======SUCCESS====== 2022-01-04 02:04:26.548839 ===Row: 144 Success: True

{'reading_1': 10.37076753, 'reading_2': 12.14934757, 'reading_3': 25.94981437, 'reading_4': 25.62703876, 'reading_5': 20.49636707, 'reading_6': 11.51899891}
======SUCCESS====== 2022-01-04 02:04:31.575473 ===Row: 145 Success: True

{'reading_1': 10.71503565, 'reading_2': 12.60543561, 'reading_3': 25.178634, 'reading_4': 25.9558296, 'reading_5': 20.96593918, 'reading_6': 12.60597666}
======SUCCESS====== 2022-01-04 02:04:36.589647 ===Row: 146 
Success: True
{'reading_1': 10.71374828, 'reading_2': 12.62617027, 'reading_3': 25.95287545, 'reading_4': 25.79256005, 'reading_5': 20.20300859, 'reading_6': 11.12760022}
======SUCCESS====== 2022-01-04 02:04:41.615753 ===Row: 147 Success: True

{'reading_1': 10.80528358, 'reading_2': 12.80008492, 'reading_3': 25.91586625, '

...

{'reading_1': 10.54654148, 'reading_2': 10.60748047, 'reading_3': 25.34228713, 'reading_4': 25.63074918, 'reading_5': 30.85596362, 'reading_6': 11.88161331}
======SUCCESS====== 2022-01-04 03:11:10.750128 ===Row: 941 Success: True

{'reading_1': 10.14692796, 'reading_2': 10.34183612, 'reading_3': 25.15975432, 'reading_4': 25.37574162, 'reading_5': 30.62738267, 'reading_6': 12.44105733}
======SUCCESS====== 2022-01-04 03:11:15.777040 ===Row: 942 Success: True

{'reading_1': 10.82606382, 'reading_2': 10.51566749, 'reading_3': 25.06667212, 'reading_4': 25.25339095, 'reading_5': 30.7652516, 'reading_6': 11.66272026}
======SUCCESS====== 2022-01-04 03:11:20.802354 ===Row: 943 Success: True

{'reading_1': 10.64427996, 'reading_2': 10.446795, 'reading_3': 25.33857697, 'reading_4': 25.7403992, 'reading_5': 30.85077565, 'reading_6': 12.2396369}
======SUCCESS====== 2022-01-04 03:11:25.804316 ===Row: 944 Success: True

{'reading_1': 10.97726768, 'reading_2': 10.39637622, 'reading_3': 25.16471814, 'r

{'reading_1': 10.53309896, 'reading_2': 11.49108254, 'reading_3': 22.91861155, 'reading_4': 22.93982242, 'reading_5': 30.71337257, 'reading_6': 11.79059931}
======SUCCESS====== 2022-01-04 03:14:11.627827 ===Row: 977 Success: True

{'reading_1': 10.71248276, 'reading_2': 11.10577165, 'reading_3': 22.92000967, 'reading_4': 22.06431885, 'reading_5': 30.613278700000002, 'reading_6': 12.42541408}
======SUCCESS====== 2022-01-04 03:14:16.652111 ===Row: 978 Success: True

{'reading_1': 10.98196687, 'reading_2': 11.2023066, 'reading_3': 22.58886969, 'reading_4': 22.94508504, 'reading_5': 30.95302761, 'reading_6': 11.93025057}
======SUCCESS====== 2022-01-04 03:14:21.678584 ===Row: 979 Success: True

{'reading_1': 10.36888101, 'reading_2': 11.55435575, 'reading_3': 21.45356413, 'reading_4': 21.58893113, 'reading_5': 30.22639891, 'reading_6': 12.25640536}
======SUCCESS====== 2022-01-04 03:14:26.702161 ===Row: 980 Success: True

{'reading_1': 10.02176362, 'reading_2': 11.21472199, 'reading_3': 21.8

{'reading_1': 10.75069814, 'reading_2': 11.33528846, 'reading_3': 20.6285625, 'reading_4': 20.80478133, 'reading_5': 20.29866568, 'reading_6': 11.82586626}
======SUCCESS====== 2022-01-04 03:17:12.462686 ===Row: 1013 Success: True

{'reading_1': 10.81541729, 'reading_2': 11.83971116, 'reading_3': 20.95172463, 'reading_4': 20.00970745, 'reading_5': 20.57408218, 'reading_6': 12.29213774}

======SUCCESS====== 2022-01-04 03:17:17.489024 ===Row: 1014 Success: True
{'reading_1': 10.64325734, 'reading_2': 11.94034253, 'reading_3': 20.05399943, 'reading_4': 20.53600225, 'reading_5': 20.4000397, 'reading_6': 11.64632362}
======SUCCESS====== 2022-01-04 03:17:22.515491 ===Row: 1015 Success: True

{'reading_1': 10.26179831, 'reading_2': 11.89439302, 'reading_3': 20.79625505, 'reading_4': 20.63385495, 'reading_5': 20.20707105, 'reading_6': 12.23672471}
======SUCCESS====== 2022-01-04 03:17:27.542349 ===Row: 1016 Success: True

{'reading_1': 10.03429322, 'reading_2': 11.95877729, 'reading_3': 20.46502

{'reading_1': 10.18377834, 'reading_2': 13.04307804, 'reading_3': 20.74487692, 'reading_4': 20.78267975, 'reading_5': 20.42105831, 'reading_6': 11.7238748}
======SUCCESS====== 2022-01-04 03:20:13.320682 ===Row: 1049 Success: True

{'reading_1': 10.96816014, 'reading_2': 12.99154459, 'reading_3': 21.62745565, 'reading_4': 21.29397096, 'reading_5': 20.64185293, 'reading_6': 12.25200172}
======SUCCESS====== 2022-01-04 03:20:18.347041 ===Row: 1050 Success: True

{'reading_1': 10.09171376, 'reading_2': 13.05924698, 'reading_3': 21.11299262, 'reading_4': 21.74954799, 'reading_5': 20.46292602, 'reading_6': 11.61437012}
======SUCCESS====== 2022-01-04 03:20:23.373637 ===Row: 1051 Success: True

{'reading_1': 10.3841571, 'reading_2': 13.30724032, 'reading_3': 21.77096367, 'reading_4': 21.110705600000003, 'reading_5': 20.98199781, 'reading_6': 12.00949679}
======SUCCESS====== 2022-01-04 03:20:28.400029 ===Row: 1052 Success: True

{'reading_1': 10.74583461, 'reading_2': 13.28654255, 'reading_3': 2

2022-01-04 03:22:49,047   wiotp.sdk.device.client.DeviceClient  INFO    Disconnected from the IBM Watson IoT Platform
2022-01-04 03:22:49,051   wiotp.sdk.device.client.DeviceClient  INFO    Closed connection to the IBM Watson IoT Platform


In [8]:
deviceCli.disconnect()

2021-12-07 10:05:20,282   wiotp.sdk.device.client.DeviceClient  INFO    Disconnected from the IBM Watson IoT Platform
2021-12-07 10:05:20,285   wiotp.sdk.device.client.DeviceClient  INFO    Closed connection to the IBM Watson IoT Platform
2021-12-07 10:05:22,037   wiotp.sdk.device.client.DeviceClient  INFO    Connected successfully: d:YOUR_ORGID:YOUR_Device:YOUR_Device001
